In [1]:
from data201 import db_connection
import csv
from pandas import DataFrame

In [2]:
conn = db_connection(config_file = 'team_project.ini')
cursor = conn.cursor()

## Create Tables

In [3]:
cursor.execute("ALTER TABLE administrator DROP FOREIGN KEY fk_administrator_school;")
cursor.execute("DROP TABLE IF EXISTS attendance;")
cursor.execute("DROP TABLE IF EXISTS grade_details;")
cursor.execute("DROP TABLE IF EXISTS takes;")
cursor.execute("DROP TABLE IF EXISTS teaches;")
cursor.execute("DROP TABLE IF EXISTS course;")
cursor.execute("DROP TABLE IF EXISTS guardian_student_relationship;")
cursor.execute("DROP TABLE IF EXISTS guardian;")
cursor.execute("DROP TABLE IF EXISTS student;")
cursor.execute("DROP TABLE IF EXISTS teacher;")
cursor.execute("DROP TABLE IF EXISTS school;")
cursor.execute("DROP TABLE IF EXISTS district;")
cursor.execute("DROP TABLE IF EXISTS administrator;")
cursor.execute("DROP TABLE IF EXISTS users;")
conn.commit()

create_table_query = [
    """
    CREATE TABLE users (
        user_id INT PRIMARY KEY AUTO_INCREMENT,
        username VARCHAR(50) NOT NULL UNIQUE,
        password VARCHAR(256) NOT NULL,
        plain_password VARCHAR(256) NOT NULL,
        email VARCHAR(100) NOT NULL UNIQUE,
        role ENUM('teacher', 'student', 'guardian', 'school_admin', 'district_admin') NOT NULL
    );
    """,
    """
    CREATE TABLE administrator (
        administrator_id INT PRIMARY KEY AUTO_INCREMENT,
        user_id INT NOT NULL,
        school_id INT,
        first_name VARCHAR(50) NOT NULL,
        middle_name VARCHAR(50),
        last_name VARCHAR(50) NOT NULL,
        phone_number VARCHAR(20),
        employment_type ENUM('full-time', 'part-time') NOT NULL,
        salary DECIMAL(10, 2),
        join_date DATE,
        supervisor_id INT,
        FOREIGN KEY (user_id) REFERENCES users(user_id),
        FOREIGN KEY (supervisor_id) REFERENCES administrator(administrator_id)
    );
    """,
    """
    CREATE TABLE district (
        district_id INT PRIMARY KEY AUTO_INCREMENT,
        name VARCHAR(100) NOT NULL,
        superintendent_id INT,
        FOREIGN KEY (superintendent_id) REFERENCES administrator(administrator_id)
    );
    """,
    """
    CREATE TABLE school (
        school_id INT PRIMARY KEY AUTO_INCREMENT,
        district_id INT NOT NULL,
        name VARCHAR(100) NOT NULL,
        school_type ENUM('elementary', 'middle', 'high', 'special_ed') NOT NULL,
        address VARCHAR(255),
        city VARCHAR(50),
        state VARCHAR(20),
        zip VARCHAR(10),
        principal_id INT,
        FOREIGN KEY (district_id) REFERENCES district(district_id),
        FOREIGN KEY (principal_id) REFERENCES administrator(administrator_id)
    );
    """,
    """
    CREATE TABLE teacher (
        teacher_id INT PRIMARY KEY AUTO_INCREMENT,
        user_id INT NOT NULL,
        school_id INT NOT NULL,
        first_name VARCHAR(50) NOT NULL,
        middle_name VARCHAR(50),
        last_name VARCHAR(50) NOT NULL,
        phone_number VARCHAR(20),
        employment_type ENUM('full-time', 'part-time', 'substitute') NOT NULL,
        salary DECIMAL(10, 2),
        join_date DATE,
        FOREIGN KEY (user_id) REFERENCES users(user_id),
        FOREIGN KEY (school_id) REFERENCES school(school_id)
    );
    """,
    """
    CREATE TABLE student (
        student_id INT PRIMARY KEY AUTO_INCREMENT,
        user_id INT NOT NULL,
        school_id INT NOT NULL,
        first_name VARCHAR(50) NOT NULL,
        middle_name VARCHAR(50),
        last_name VARCHAR(50) NOT NULL,
        date_of_birth DATE,
        grade_level VARCHAR(4) NOT NULL,
        homeroom_id INT,
        homeroom_teacher_id INT,
        FOREIGN KEY (user_id) REFERENCES users(user_id),
        FOREIGN KEY (school_id) REFERENCES school(school_id),
        FOREIGN KEY (homeroom_teacher_id) REFERENCES teacher(teacher_id)
    );
    """,
    """
    CREATE TABLE guardian (
        guardian_id INT PRIMARY KEY AUTO_INCREMENT,
        user_id INT NOT NULL,
        first_name VARCHAR(50) NOT NULL,
        middle_name VARCHAR(50),
        last_name VARCHAR(50) NOT NULL,
        phone_number VARCHAR(20),
        FOREIGN KEY (user_id) REFERENCES users(user_id)
    );
    """,
    """
    CREATE TABLE guardian_student_relationship (
        guardian_id INT NOT NULL,
        student_id INT NOT NULL,
        relationship ENUM('mother', 'father', 'grandmother', 'grandfather', 'others') NOT NULL,
        PRIMARY KEY (guardian_id, student_id),
        FOREIGN KEY (guardian_id) REFERENCES guardian(guardian_id),
        FOREIGN KEY (student_id) REFERENCES student(student_id)
    );
    """,
    """
    CREATE TABLE course (
        course_id INT NOT NULL,
        name VARCHAR(10) NOT NULL,
        PRIMARY KEY (course_id)
    );
    """,
    """
    CREATE TABLE teaches (
        teacher_id INT,
        course_id INT,
        day VARCHAR(10),
        start_time TIME,
        end_time TIME,
        homeroom_id INT,
        school_id INT,
        grade_level VARCHAR(5),
        PRIMARY KEY (teacher_id, course_id, day, start_time),
        FOREIGN KEY (teacher_id) REFERENCES teacher(teacher_id),
        FOREIGN KEY (course_id) REFERENCES course(course_id)
    );
    """,
    """
    CREATE TABLE takes (
        student_id INT NOT NULL,
        course_id INT NOT NULL,
        day VARCHAR(10) NOT NULL,
        start_time TIME,
        end_time TIME,
        PRIMARY KEY (student_id, day, start_time),
        FOREIGN KEY (student_id) REFERENCES student(student_id),
        FOREIGN KEY (course_id) REFERENCES course(course_id)
    );
    """,
    """
    CREATE TABLE grade_details (
        student_id INT NOT NULL,
        course_id INT NOT NULL,
        grade_type ENUM('homework1','homework2', 'quiz', 'mid exam', 'final exam') NOT NULL,
        score INT NOT NULL,
        weight DECIMAL(3, 2) NOT NULL,
        PRIMARY KEY (student_id, course_id, grade_type),
        FOREIGN KEY (student_id) REFERENCES student(student_id),
        FOREIGN KEY (course_id) REFERENCES course(course_id)
    );
    """,
    """
    CREATE TABLE attendance (
        student_id INT NOT NULL,
        date DATE NOT NULL,
        status ENUM('present', 'absent', 'late', 'excused') NOT NULL,
        recorded_by INT NOT NULL,
        notes TEXT,
        PRIMARY KEY (student_id, date),
        FOREIGN KEY (student_id) REFERENCES student(student_id),
        FOREIGN KEY (recorded_by) REFERENCES teacher(teacher_id)
    );
    """
]

In [4]:
for query in create_table_query:
    cursor.execute(query)
conn.commit()

## Clear existing data from all tables

In [5]:
cursor.execute("DELETE FROM users;")
cursor.execute("DELETE FROM district;")
cursor.execute("DELETE FROM school;")
cursor.execute("DELETE FROM teacher;")
cursor.execute("DELETE FROM student;")
cursor.execute("DELETE FROM guardian;")
cursor.execute("DELETE FROM guardian_student_relationship;")
cursor.execute("DELETE FROM administrator;")
cursor.execute("DELETE FROM course;")
cursor.execute("DELETE FROM teaches;")
cursor.execute("DELETE FROM takes;")
cursor.execute("DELETE FROM grade_details;")
cursor.execute("DELETE FROM attendance;")
conn.commit()

## SQL to load the tables

In [6]:
users_insert = (  
    """
    INSERT INTO users
    VALUES (%s, %s, %s, %s, %s, %s)
    """
)

district_insert = (  
    """
    INSERT INTO district
    VALUES (%s, %s, %s)
    """
)

school_insert = (  
    """
    INSERT INTO school
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
)

teacher_insert = (  
    """
    INSERT INTO teacher
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
)

student_insert = (  
    """
    INSERT INTO student
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
)

guardian_insert = (  
    """
    INSERT INTO guardian
    VALUES (%s, %s, %s, %s, %s, %s)
    """
)

guardian_student_relationship_insert = (  
    """
    INSERT INTO guardian_student_relationship
    VALUES (%s, %s, %s)
    """
)

administrator_insert = (  
    """
    INSERT INTO administrator
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
)

course_insert = (  
    """
    INSERT INTO course
    VALUES (%s, %s)
    """
)

teaches_insert = (  
    """
    INSERT INTO teaches
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
)

takes_insert = (  
    """
    INSERT INTO takes
    VALUES (%s, %s, %s, %s, %s)
    """
)

grade_details_insert = (  
    """
    INSERT INTO grade_details
    VALUES (%s, %s, %s, %s, %s)
    """
)

attendance_insert = (  
    """
    INSERT INTO attendance
    VALUES (%s, %s, %s, %s, %s)
    """
)

## Insert Data Function

In [7]:
def refine_row(row):
    for i in range(len(row)):
        if row[i] == '': row[i] = None
    return row

In [8]:
def insert_data_from_csv(csv_file_path, insert_query, skip_header=True, row_slice=None):
    with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        if skip_header:
            next(reader)
        for row in reader:
            if row_slice:
                row = [row[i] for i in row_slice]
            refine_row(row)
            cursor.execute(insert_query, row)
    conn.commit()

## Insert Data

In [9]:
def insert_users():
    insert_data_from_csv("all_users_with_passwords.csv", users_insert, row_slice=[0,1,2,3,4,5])

def insert_administrators():
    insert_data_from_csv("admins_data.csv", administrator_insert)

def insert_districts():
    insert_data_from_csv("districts_data.csv", district_insert)

def insert_schools():
    insert_data_from_csv("schools_data.csv", school_insert)

def insert_teachers():
    insert_data_from_csv("teachers_data.csv", teacher_insert)

def insert_students():
    insert_data_from_csv("students_data.csv", student_insert)

def insert_guardians():
    insert_data_from_csv("guardians_data.csv", guardian_insert)

def insert_guardian_student_relationships():
    insert_data_from_csv("guardian_student_relationships.csv", guardian_student_relationship_insert)

def insert_courses():
    insert_data_from_csv("school_courses.csv", course_insert)

def insert_teaches():
    insert_data_from_csv("teaches_data.csv", teaches_insert)

def insert_takes():
    insert_data_from_csv("takes_data.csv", takes_insert)

def insert_grade_details():
    insert_data_from_csv("grade_details.csv", grade_details_insert)

def insert_attendance():
    insert_data_from_csv("attendance_data.csv", attendance_insert, row_slice=[1,2,3,4,5])

def insert_all_data():
    insert_users()
    insert_administrators()
    insert_districts()
    insert_schools()
    insert_teachers()
    insert_students()
    insert_guardians()
    insert_guardian_student_relationships()
    insert_courses()
    insert_teaches()
    insert_takes()
    insert_grade_details()
    insert_attendance()

if __name__ == "__main__":
    insert_all_data()

In [10]:
cursor.execute("""
    ALTER TABLE administrator 
    ADD CONSTRAINT fk_administrator_school 
    FOREIGN KEY (school_id) REFERENCES school(school_id);
""")
conn.commit()

In [11]:
cursor.close()
conn.close()